In [7]:
from src.constant import *
from src.configuration.configuration import Configuration

import os
from kaggle.api.kaggle_api_extended import KaggleApi

config_info = Configuration(config_file_path='/Users/vambat1/Documents/Projects/Machine-Learning/04-Movie-Recommendation-System/config/config.yaml')
#config_info.data_ingestion_config()
data_ingestion_config_info = config_info[DATA_INGESTION_CONFIG_KEY]
print(data_ingestion_config_info[DATA_INGESTION_KAGGLE_DATASET_URL_KEY])


api = KaggleApi()
api.authenticate()
dataset_url = data_ingestion_config_info[DATA_INGESTION_KAGGLE_DATASET_URL_KEY]
download_path = os.path.join(config_info[DATA_DIR_KEY], data_ingestion_config_info[DATA_INGESTION_ZIP_DATA_DIR_KEY])
api.dataset_download_files(dataset=dataset_url, path=download_path, unzip=False)

TypeError: 'Configuration' object is not subscriptable

In [6]:
file_path = "/Users/vambat1/Documents/Projects/Machine-Learning/04-Movie-Recommendation-System/notebooks/"
import zipfile
with zipfile.ZipFile("imdb-movies-dataset.zip","r") as zip_ref:
    zip_ref.extractall(file_path)

FileNotFoundError: [Errno 2] No such file or directory: 'imdb-movies-dataset.zip'

In [38]:
os.getcwd()

'/Users/vambat1/Documents/Projects/Machine-Learning/04-Movie-Recommendation-System'

In [2]:
import os
os.chdir("/Users/vambat1/Documents/Projects/Machine-Learning/04-Movie-Recommendation-System/")

In [9]:
print(config)